In [1]:
!git clone https://github.com/khandelwalsumit/TopicModelling.git

Cloning into 'TopicModelling'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 1), reused 2 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [2]:
import pandas as pd

In [3]:
data = pd.read_csv("TopicModelling/abcnews-date-text.csv")

In [5]:
documents = data[['headline_text']]
documents['index'] = documents.index

In [6]:
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [10]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [12]:
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['ratepayers', 'group', 'wants', 'compulsory', 'local', 'govt', 'voting']


 tokenized and lemmatized document: 
['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']


In [13]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [14]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [15]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [16]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(162, 1), (240, 1), (292, 1), (589, 1), (838, 1), (3570, 1), (3571, 1)]

In [17]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], dictionary[bow_doc_4310[i][0]], bow_doc_4310[i][1]))

Word 162 ("govt") appears 1 time.
Word 240 ("group") appears 1 time.
Word 292 ("vote") appears 1 time.
Word 589 ("local") appears 1 time.
Word 838 ("want") appears 1 time.
Word 3570 ("compulsori") appears 1 time.
Word 3571 ("ratepay") appears 1 time.


In [18]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5842699484464488),
 (1, 0.38798859072167835),
 (2, 0.5008422243250992),
 (3, 0.5071987254965034)]


In [19]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [20]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.018*"health" + 0.018*"border" + 0.015*"australia" + 0.013*"interview" + 0.011*"coronavirus" + 0.011*"say" + 0.010*"talk" + 0.009*"quarantin" + 0.009*"weather" + 0.008*"mental"
Topic: 1 
Words: 0.044*"trump" + 0.029*"case" + 0.029*"charg" + 0.028*"court" + 0.023*"murder" + 0.019*"face" + 0.017*"alleg" + 0.015*"year" + 0.015*"jail" + 0.014*"woman"
Topic: 2 
Words: 0.049*"coronavirus" + 0.042*"covid" + 0.015*"tasmania" + 0.012*"dead" + 0.011*"fund" + 0.010*"student" + 0.010*"break" + 0.009*"travel" + 0.009*"indigen" + 0.009*"communiti"
Topic: 3 
Words: 0.040*"queensland" + 0.032*"victoria" + 0.028*"melbourn" + 0.020*"coast" + 0.014*"farmer" + 0.014*"australia" + 0.014*"coronavirus" + 0.013*"win" + 0.012*"gold" + 0.012*"west"
Topic: 4 
Words: 0.022*"home" + 0.022*"kill" + 0.022*"news" + 0.019*"state" + 0.015*"perth" + 0.015*"peopl" + 0.014*"bushfir" + 0.012*"child" + 0.011*"care" + 0.011*"abus"
Topic: 5 
Words: 0.021*"open" + 0.020*"adelaid" + 0.016*"final" + 0.015*"tria

In [21]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.015*"countri" + 0.015*"coronavirus" + 0.011*"hour" + 0.010*"restrict" + 0.009*"covid" + 0.008*"care" + 0.008*"thursday" + 0.007*"updat" + 0.007*"age" + 0.005*"vaccin"
Topic: 1 Word: 0.013*"drum" + 0.009*"tuesday" + 0.008*"commiss" + 0.007*"violenc" + 0.007*"royal" + 0.006*"social" + 0.006*"domest" + 0.005*"septemb" + 0.005*"fiji" + 0.005*"coronavirus"
Topic: 2 Word: 0.018*"charg" + 0.018*"polic" + 0.015*"murder" + 0.011*"woman" + 0.011*"alleg" + 0.011*"court" + 0.009*"death" + 0.009*"jail" + 0.008*"sentenc" + 0.008*"assault"
Topic: 3 Word: 0.026*"trump" + 0.015*"donald" + 0.008*"weather" + 0.008*"queensland" + 0.007*"friday" + 0.007*"andrew" + 0.007*"coronavirus" + 0.007*"wall" + 0.007*"sport" + 0.006*"hill"
Topic: 4 Word: 0.019*"news" + 0.011*"australia" + 0.010*"market" + 0.009*"rural" + 0.008*"coronavirus" + 0.008*"world" + 0.007*"australian" + 0.007*"financ" + 0.006*"cricket" + 0.006*"nation"
Topic: 5 Word: 0.010*"govern" + 0.006*"coronavirus" + 0.006*"monday" + 0.

In [22]:
processed_docs[4310]

['ratepay', 'group', 'want', 'compulsori', 'local', 'govt', 'vote']

In [23]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8874972462654114	 
Topic: 0.026*"elect" + 0.021*"govern" + 0.020*"say" + 0.015*"chang" + 0.011*"labor" + 0.011*"speak" + 0.010*"drum" + 0.010*"andrew" + 0.009*"feder" + 0.009*"public"

Score: 0.012501540593802929	 
Topic: 0.049*"coronavirus" + 0.042*"covid" + 0.015*"tasmania" + 0.012*"dead" + 0.011*"fund" + 0.010*"student" + 0.010*"break" + 0.009*"travel" + 0.009*"indigen" + 0.009*"communiti"

Score: 0.01250060461461544	 
Topic: 0.032*"coronavirus" + 0.021*"nation" + 0.018*"south" + 0.017*"north" + 0.017*"restrict" + 0.015*"rise" + 0.015*"victorian" + 0.015*"water" + 0.012*"offic" + 0.012*"price"

Score: 0.012500581331551075	 
Topic: 0.040*"australia" + 0.021*"world" + 0.019*"market" + 0.017*"record" + 0.017*"protest" + 0.015*"australian" + 0.015*"china" + 0.014*"busi" + 0.013*"time" + 0.010*"million"

Score: 0.012500009499490261	 
Topic: 0.058*"polic" + 0.026*"donald" + 0.025*"live" + 0.016*"death" + 0.015*"crash" + 0.015*"investig" + 0.013*"die" + 0.013*"royal" + 0.013*"mor

In [24]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.7322425842285156	 
Topic: 0.010*"govern" + 0.006*"coronavirus" + 0.006*"monday" + 0.006*"elect" + 0.005*"feder" + 0.005*"fund" + 0.005*"plan" + 0.005*"council" + 0.005*"chang" + 0.004*"health"

Score: 0.1677512377500534	 
Topic: 0.026*"trump" + 0.015*"donald" + 0.008*"weather" + 0.008*"queensland" + 0.007*"friday" + 0.007*"andrew" + 0.007*"coronavirus" + 0.007*"wall" + 0.007*"sport" + 0.006*"hill"

Score: 0.012501309625804424	 
Topic: 0.013*"drum" + 0.009*"tuesday" + 0.008*"commiss" + 0.007*"violenc" + 0.007*"royal" + 0.006*"social" + 0.006*"domest" + 0.005*"septemb" + 0.005*"fiji" + 0.005*"coronavirus"

Score: 0.012501031160354614	 
Topic: 0.010*"live" + 0.009*"stori" + 0.007*"christma" + 0.007*"michael" + 0.006*"david" + 0.006*"know" + 0.005*"coronavirus" + 0.005*"june" + 0.005*"australia" + 0.005*"search"

Score: 0.012500966899096966	 
Topic: 0.015*"countri" + 0.015*"coronavirus" + 0.011*"hour" + 0.010*"restrict" + 0.009*"covid" + 0.008*"care" + 0.008*"thursday" + 0.007*"u

In [25]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.48525750637054443	 Topic: 0.040*"australia" + 0.021*"world" + 0.019*"market" + 0.017*"record" + 0.017*"protest"
Score: 0.21474039554595947	 Topic: 0.026*"elect" + 0.021*"govern" + 0.020*"say" + 0.015*"chang" + 0.011*"labor"
Score: 0.18333332240581512	 Topic: 0.040*"queensland" + 0.032*"victoria" + 0.028*"melbourn" + 0.020*"coast" + 0.014*"farmer"
Score: 0.016667794436216354	 Topic: 0.032*"coronavirus" + 0.021*"nation" + 0.018*"south" + 0.017*"north" + 0.017*"restrict"
Score: 0.016667651012539864	 Topic: 0.018*"health" + 0.018*"border" + 0.015*"australia" + 0.013*"interview" + 0.011*"coronavirus"
Score: 0.01666666753590107	 Topic: 0.044*"trump" + 0.029*"case" + 0.029*"charg" + 0.028*"court" + 0.023*"murder"
Score: 0.01666666753590107	 Topic: 0.049*"coronavirus" + 0.042*"covid" + 0.015*"tasmania" + 0.012*"dead" + 0.011*"fund"
Score: 0.01666666753590107	 Topic: 0.022*"home" + 0.022*"kill" + 0.022*"news" + 0.019*"state" + 0.015*"perth"
Score: 0.01666666753590107	 Topic: 0.021*"ope

In [26]:
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

Score: 0.3214333653450012	 Topic: 0.010*"live" + 0.009*"stori" + 0.007*"christma" + 0.007*"michael" + 0.006*"david"
Score: 0.2841341197490692	 Topic: 0.008*"wednesday" + 0.007*"cattl" + 0.006*"export" + 0.005*"georg" + 0.005*"price"
Score: 0.2777499258518219	 Topic: 0.011*"kill" + 0.009*"crash" + 0.008*"dead" + 0.007*"korea" + 0.006*"border"
Score: 0.016671141609549522	 Topic: 0.013*"drum" + 0.009*"tuesday" + 0.008*"commiss" + 0.007*"violenc" + 0.007*"royal"
Score: 0.01666976697742939	 Topic: 0.010*"govern" + 0.006*"coronavirus" + 0.006*"monday" + 0.006*"elect" + 0.005*"feder"
Score: 0.01666972227394581	 Topic: 0.026*"trump" + 0.015*"donald" + 0.008*"weather" + 0.008*"queensland" + 0.007*"friday"
Score: 0.016669126227498055	 Topic: 0.015*"countri" + 0.015*"coronavirus" + 0.011*"hour" + 0.010*"restrict" + 0.009*"covid"
Score: 0.01666801981627941	 Topic: 0.014*"interview" + 0.010*"morrison" + 0.010*"scott" + 0.007*"mental" + 0.007*"peter"
Score: 0.01666792295873165	 Topic: 0.019*"news" +